In [8]:
import torch 
import pandas as pd 
import pickle
import numpy as np
import os
import sys 

from torch.utils.data import Dataset, DataLoader
from torch import nn


import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from tqdm import trange

sns.set_theme()

In [9]:
# Load data
freq_data = np.genfromtxt("TTF_Data_100.000_1.500_2.500.csv.csv")

# Load pickle
with open("TTF_data.pt", "rb") as f:
    TTF_data = pickle.load(f)

with open("Vs_values.pt", "rb") as f:
    Vs_data = pickle.load(f)

TTF_data = np.array(TTF_data)
Vs_data = np.array(Vs_data)

# Get Vs_data less than 800
index = Vs_data < 800
TTF_data = TTF_data[index]
Vs_data = Vs_data[index]

print(TTF_data.shape, Vs_data.shape)

# Use only certain freq
idx = 0
freq = freq_data[:]

(146, 256) (146,)


In [17]:
# Separate the 20 values randomly for testing
test_indices = np.random.choice(len(TTF_data), 20, replace=False)
test_TTF_data = TTF_data[test_indices][:, :]
test_Vs_data = Vs_data[test_indices]

print(test_TTF_data.shape, test_Vs_data.shape)

(20, 256) (20,)


In [11]:
class TTFDataset(Dataset):
    def __init__(self, TTF_data, Vs_array, freq_array):
        self.TTF_data = TTF_data
        self.Vs_array = Vs_array
        self.freq_array = freq_array

    def __len__(self):
        return len(self.TTF_data)

    def __getitem__(self, idx):

        # Input 
        Vs = self.Vs_array[idx] * np.ones(30)
        freq = self.freq_array[idx] * np.ones(1)
        input_data = np.concatenate((Vs, freq))
        input_data = torch.tensor(input_data, dtype=torch.float32)

        # Target
        TTF = self.TTF_data[idx] * np.ones(1)
        target = torch.tensor(TTF, dtype=torch.float32)

        return input_data, target

In [13]:
# Crate Meshgrid
freq_values, Vs_values = np.meshgrid(freq, train_Vs_data)
print(Vs_values.shape, freq_values.shape, train_TTF_data.shape)

# Flatten the meshgrid
Vs_values = Vs_values.flatten()
freq_values = freq_values.flatten()
TTF_values = train_TTF_data.flatten()
print(Vs_values.shape, freq_values.shape, TTF_values.shape)

(136, 256) (136, 256) (136, 256)
(34816,) (34816,) (34816,)


In [14]:
# Normalize the data
def normalize_data(data, mean, std):
    return (data - mean) / std

def denormalize_data(data, mean, std):
    return data * std + mean

# Normalize the data
mean_Vs = Vs_values.mean()
std_Vs = Vs_values.std()
mean_freq = freq_values.mean()
std_freq = freq_values.std()
mean_TTF = TTF_values.mean()
std_TTF = TTF_values.std()

Vs_values = normalize_data(Vs_values, mean_Vs, std_Vs)
freq_values = normalize_data(freq_values, mean_freq, std_freq)
TTF_values = normalize_data(TTF_values, mean_TTF, std_TTF)

# Create dataset
dataset = TTFDataset(TTF_values, Vs_values, freq_values)